In [32]:
import pandas as pd
import numpy as np
import re # Used for regular expressions in parsing

In [ ]:
# --- 1. Data Ingestion ---
# Read the raw workout data from two separate text files.
# .strip() removes leading/trailing whitespace, and `if line.strip()` ignores empty lines.
with open('Data/Strong.txt', 'r', encoding='utf-8') as f1, open('Data/strongv6.txt', 'r', encoding='utf-8') as f2:
    strong_lines = [line.strip() for line in f1 if line.strip()]
    strongv6_lines = [line.strip() for line in f2 if line.strip()]

In [ ]:
# --- 2. Helper Function: Group Lines by Date ---
# This function groups the raw text lines into sublists, where each sublist
# represents a single workout session, starting with a date.
def group_by_date(lines):
    grouped = [] # List to hold all workout groups
    current = [] # List to hold lines for the current workout
    for line in lines:
        # If a line starts with a digit, it's assumed to be a new date,
        # indicating the start of a new workout session.
        if line and line[0].isdigit():
            if current: # If 'current' is not empty, it means a previous workout group is complete
                grouped.append(current) # Add the completed group to 'grouped'
            current = [line] # Start a new group with the current date line
        else:
            # If the line doesn't start with a digit, it's an exercise entry
            # for the current workout, so append it to the 'current' group.
            current.append(line)
    if current: # Add the last collected group if it's not empty
        grouped.append(current)
    return grouped

In [ ]:
# Apply the grouping function to both sets of raw lines
strong_lines_grouped = group_by_date(strong_lines)
strongv6_lines_grouped = group_by_date(strongv6_lines)

In [ ]:
# --- 3. Helper Function: Parse Exercise Lines ---
# This function takes a single exercise line (e.g., 'Bench 50kgx12 55kgx10')
# and parses it into a structured format (list of dictionaries).
def parse_exercise_line(line):
    """
    Parses lines like:
    - 'Bench 50kgx12 55kgx10 60kgx9'
    - 'Squads 80kgx8x8'
    - 'Pull ups 10x8x8' (where weight is implied/bodyweight, so 0kg or None)
    into a list of dictionaries with 'exercise', 'weight', 'reps', and 'order' (set number).
    """
    parts = line.split()
    if not parts: # Handle empty lines that might slip through
        return []

    exercise_name_parts = []
    sets_data_parts = []
    
    # Separate exercise name from sets data
    # Iterate through parts to find where the weight/reps pattern starts
    found_sets_start = False
    for part in parts:
        if re.match(r'(\d+kgx\d+.*)|(\d+x\d+.*)', part): # Check for weightxreps or repsxreps pattern
            found_sets_start = True
        if not found_sets_start:
            exercise_name_parts.append(part)
        else:
            sets_data_parts.append(part)
            
    exercise = ' '.join(exercise_name_parts) # Reconstruct exercise name (e.g., 'Pull ups')
    
    results = []
    order = 1 # Initialize set order for each exercise within a workout
    
    for s in sets_data_parts:
        # Pattern 1: Matches 'WEIGHTkgxREPS' or 'WEIGHTkgxREPSxREPS...'
        m_kg = re.match(r'(\d+)kgx(\d+(?:x\d+)*)', s)
        if m_kg:
            weight = int(m_kg.group(1))
            reps_list = list(map(int, m_kg.group(2).split('x')))
            for reps in reps_list:
                results.append({'exercise': exercise, 'weight': weight, 'reps': reps, 'order': order})
                order += 1
        else:
            # Pattern 2: Matches 'REPSxREPS...' (e.g., for bodyweight exercises like Pull-ups)
            # This assumes if 'kg' is not present, weight is 0 or None.
            m_reps_only = re.match(r'(\d+(?:x\d+)*)', s)
            if m_reps_only:
                reps_list = list(map(int, m_reps_only.group(1).split('x')))
                for reps in reps_list:
                    # Assign weight as 0 for bodyweight exercises or when not specified
                    results.append({'exercise': exercise, 'weight': 0, 'reps': reps, 'order': order})
                    order += 1
            else:
                # Fallback for any unhandled patterns, or single numbers (e.g., '10' for 10 reps)
                # This might need refinement based on actual data variations
                try:
                    reps_val = int(s)
                    results.append({'exercise': exercise, 'weight': 0, 'reps': reps_val, 'order': order})
                    order += 1
                except ValueError:
                    # If parsing fails, skip or log an error
                    print(f"Warning: Could not parse part '{s}' in line '{line}'")
                    continue
    return results

In [ ]:
# --- 4. Helper Function: Create DataFrame from Grouped Lines ---
# This function iterates through the grouped workout data and creates a Pandas DataFrame.
def create_dataframe(grouped_lines):
    rows = []
    for group in grouped_lines:
        date_str = group[0] # The first line of each group is the date
        # Attempt to parse date, assuming DD.MM.YYYY format
        try:
            # If year is missing (e.g., '17.11'), assume current year or the year the data was collected for
            # For this project, as per the example, it seems to be 2024.
            # A more robust solution would infer the year or require it in input.
            if len(date_str.split('.')) == 2: # e.g., '17.11'
                date = pd.to_datetime(date_str + '.2024', format="%d.%m.%Y")
            else: # e.g., '13.11.2024'
                date = pd.to_datetime(date_str, format="%d.%m.%Y")
        except ValueError:
            print(f"Warning: Could not parse date '{date_str}'. Skipping group.")
            continue

        for line in group[1:]: # Iterate through exercise lines in the group
            for entry in parse_exercise_line(line):
                rows.append({'Date': date, **entry}) # Add date to each parsed exercise entry
    
    df = pd.DataFrame(rows)
    # Rename columns for clarity and consistency with Power BI
    df.columns = ['Date', 'Exercise', 'Weight (kg)', 'Reps', 'Order']
    return df

# Create DataFrames for both strong.txt and strongv6.txt
df_strong = create_dataframe(strong_lines_grouped)
df_strongv6 = create_dataframe(strongv6_lines_grouped)

array(['Pull-ups', 'Deadlifts', 'Bench', 'Knee', 'Squads',
       'Underhand Barbell Row', 'Overhead Press', 'Bulgarian Squads',
       'Leg Raises', 'Dips'], dtype=object)

In [ ]:
# --- 5. Data Cleaning and Normalization (Exercise Names) ---
# Define a mapping for inconsistent exercise names to a standardized format.
exercise_name_mapping = {
    'Pull': 'Pull-ups',
    'pull': 'Pull-ups',
    'Pull-up': 'Pull-ups',
    'Dipy': 'Dips',
    'Squars': 'Squads',
    'Squats': 'Squads', # Standardize to 'Squads' as per your original notebook
    'Deadlift': 'Deadlifts',
    'Bulgarian': 'Bulgarian Squads',
    'Leg': 'Leg Raises',
    'Overhead': 'Overhead Press',
    'Underhand': 'Underhand Barbell Row',
    'Overhand': 'Overhead Press', # Correcting potential typo for Overhead Press
    'Knee': 'Leg Raises', # Assuming 'Knee' refers to Leg Raises as per original notebook
    'Kółko': 'Wheel', # Assuming 'Kółko' refers to 'Wheel'
    'Barbell': 'Barbell Row' # Assuming 'Barbell' refers to Barbell Row
}

# Apply normalization to both DataFrames
df_strong['Exercise'] = df_strong['Exercise'].replace(exercise_name_mapping)
df_strongv6['Exercise'] = df_strongv6['Exercise'].replace(exercise_name_mapping)

# Handle cases where weight might be None (e.g., for bodyweight exercises)
# Replace None/NaN in 'Weight (kg)' with 0 for numerical calculations.
df_strong['Weight (kg)'] = df_strong['Weight (kg)'].fillna(0)
df_strongv6['Weight (kg)'] = df_strongv6['Weight (kg)'].fillna(0)

In [ ]:
# --- 6. Concatenate DataFrames ---
# Combine the two cleaned DataFrames into a single, comprehensive DataFrame.
# ignore_index=True ensures the new DataFrame has a continuous index.
big_strong = pd.concat([df_strong, df_strongv6], ignore_index=True)

# Display unique exercise names after full normalization to verify
print("Unique Exercise Names after full normalization:")
print(big_strong['Exercise'].unique())
print("-" * 80)


In [ ]:
# --- 7. Save Combined Data to Excel ---
# This Excel file will be the input for Power BI.
big_strong.to_excel('big_strong.xlsx', index=False)
print("Combined and cleaned data saved to 'big_strong.xlsx'")
print("-" * 80)